# Método de Broyden para sistemas de ecuaciones no lineales.
## Andrade Vidal Fernando adrian.

### ¿Qué es el metodo de Broyden?

Es un método que tarda más en converger pero tiene menos complejidad en sus cálculos a diferencia del método de Newton, la principal ventaja de este método es que no tenemos que calcular la inversa de la Jacobina, haciendo mucho más sencillo el calculo para algunos sistemas.

#### Características:

+ No se requiere invertir la matriz jacobiana.
+ Es necesario contar con dos aproximaciones a la solución.
+ La convergencía sera más rápida que el punto fijo pero más lenta que en el método de Newton.

El presente cuadernillo proporcionara paso a paso la solución de un sistema de ecuaciones no lineales, aunque puede ser modificado para acoplarlo a tus necesidades.


In [42]:
from sympy import * #Usaremos SymPy para computo simbólico.

import pandas as pd #Usaremos Pandas para crear y manejar data frames donde registraremos los datos.

import IPython.display as disp #Usaremos el display para visualizar ecuaciones.

Estableceremos 3 variables aunque se pueden añadir màs en esta declaración de variables.

In [43]:
#Declaramos los simbolos que usaremos para las ecuaciónes.

x,y,z = symbols('x y z')

#Estamos insertando las funciones con las que trabajaremos, notar que puede haber simplificaciones.

funcion_original_1 = 15*x + y**2 - 4*z - 13
funcion_original_2 = x**2 + 10*y - z - 22
funcion_original_3 = y**3 - 25*z + 22

#Creamos nuestra matrix de las funciones insertadas

matriz_funciones = Matrix([funcion_original_1, funcion_original_2, funcion_original_3])

#matriz_funciones = Matrix([funcion_original_1, funcion_original_2])


#Imprimimos las funciones insertadas para asegurarnos.

print("Esta es la primer ecuación original: \n")
disp.display(funcion_original_1)

print("\n\nEsta es la segunda ecuación original: \n")
disp.display(funcion_original_2)

print("\n\nEsta es la tercer ecuación original: \n")
disp.display(funcion_original_3)

print("\n\nEsta es la matrix de nuestras ecuaciones: \n")
display(matriz_funciones)

Esta es la primer ecuación original: 



15*x + y**2 - 4*z - 13



Esta es la segunda ecuación original: 



x**2 + 10*y - z - 22



Esta es la tercer ecuación original: 



y**3 - 25*z + 22



Esta es la matrix de nuestras ecuaciones: 



Matrix([
[15*x + y**2 - 4*z - 13],
[  x**2 + 10*y - z - 22],
[      y**3 - 25*z + 22]])

Encontraremos una segunda aproximación usando el metodo de Newton.

In [44]:
#Obtenemos la jacobiana de la matriz con respecto a los simbolos x,y,z

jacobiano = matriz_funciones.jacobian([x, y, z])

#jacobiano = matriz_funciones.jacobian([x, y])

display(jacobiano)

Matrix([
[ 15,    2*y,  -4],
[2*x,     10,  -1],
[  0, 3*y**2, -25]])

In [45]:
# Declaramos nuestro punto inicial y nuestro numero de iteraciones a realizar.
x_i= 2
y_i= 2
z_i= 2

numero_iteraciones = 20

#Data Frame donde guardaremos los resultados.
df = pd.DataFrame()

jacobiano_evaluado = jacobiano.subs({x:x_i,y:y_i,z:z_i})#Evaluamos el jacobiano en el punto dado.
    
jacobiano_evaluado_inverso = jacobiano_evaluado**-1 #Obtenemos la inversa del jacobiano evaluado.
matriz_funciones_evaluado = matriz_funciones.subs({x:x_i,y:y_i,z:z_i}) #Obtenemos nuestras ecuaciones evaluadas.

solucion_sistema = jacobiano_evaluado_inverso * matriz_funciones_evaluado #Resolvemos el sistema de ecuaciones.
    
matriz_solucion = Matrix([x_i, y_i, z_i]) #Convertimos a matriz nuestras soluciones
    
nueva_solucion_matriz = matriz_solucion - solucion_sistema #Resolvemos las ultimas operaciones para dar el nuevo punto
    
#Esta sera la nueva solucion de la matriz la cual estamos asignando de nuevo a las variables para repetir el proceso
    
#Hay que notar que estamos convirtiendo los numeros a flotantes de nuevo para que sea mas facil trabajar con ellos y rápido.
    
x_ii = float(nueva_solucion_matriz[0,0])
y_ii = float(nueva_solucion_matriz[1,0])
z_ii = float(nueva_solucion_matriz[2,0])

Ahora empezamos a utilizar el metodo de Broyden en este punto porque ya aplicamos newton para encontrar otra aproximación.

In [46]:
iteraciones = 0

while iteraciones <= numero_iteraciones:
    
    #Empezamos obteniendo el valor de s
    si = Matrix([x_i,y_i,z_i]) - Matrix([x_ii, y_ii, z_ii])


    #Ahora encontramos Yi

    yi = matriz_funciones.subs({x:x_i,y:y_i,z:z_i}) - matriz_funciones.subs({x:x_ii,y:y_ii,z:z_ii})


    #obtenemos nuestros parametros necesarios

    p = Transpose(si) * jacobiano_evaluado_inverso

    u = si - (jacobiano_evaluado_inverso * yi)

    v = u*p

    w = Transpose(si) * jacobiano_evaluado_inverso * yi

    jacobiano_evaluado_inverso = jacobiano_evaluado_inverso + (v / w[0,0])

    nueva_solucion_broy = Matrix([x_ii,y_ii,z_ii]) -(jacobiano_evaluado_inverso * matriz_funciones.subs({x:x_ii,y:y_ii,z:z_ii}))



    x_i = x_ii
    y_i = y_ii
    z_i = z_ii

    x_ii = nueva_solucion_broy[0,0]
    y_ii = nueva_solucion_broy[1,0]
    z_ii = nueva_solucion_broy[2,0]

    df = df.append({
                'xi':x_i,
                'yi':y_i,
                'zi':z_i,
                'xii':x_ii,
                'yii':y_ii,
                'zii':z_ii,
                'f1_original':float(funcion_original_1.subs({x:x_ii,y:y_ii,z:z_ii})), #Se evalua la funcion 1 original en el punto dado
                'f2_original':float(funcion_original_2.subs({x:x_ii,y:y_ii,z:z_ii})), #Se evalua la funcion 2 original en el punto dado
                'f3_original':float(funcion_original_3.subs({x:x_ii,y:y_ii,z:z_ii}))}, #Se evalua la funcion 3 original en el punto dado
                ignore_index=True #Ignorar este parametro.
                 )

    iteraciones += 1
    
disp.display(df)


,f1_original,f2_original,f3_original,xi,xii,yi,yii,zi,zii
0,-8.273911e-02,-1.335083e-03,-5.378805e-01,0.865556,0.884474831194832,2.392623,2.25782815811926,1.388459,1.36191239135717
1,1.866251e-03,-9.743255e-03,1.244941e-02,0.884474831194832,0.884697704607021,2.25782815811926,2.25454610847883,1.36191239135717,1.33789436825777
2,-1.110365e-04,-1.343416e-04,-7.723200e-04,0.884697704607021,0.884554254440435,2.25454610847883,2.25565280351807,1.33789436825777,1.33909860579149
3,-2.409831e-06,-2.030987e-06,-1.671552e-05,0.884554254440435,0.884551874862766,2.25565280351807,2.25566412433337,1.33909860579149,1.33907529364429
4,3.992547e-09,3.331689e-09,2.770338e-08,0.884551874862766,0.884551836136517,2.25566412433337,2.25566427696241,1.33907529364429,1.33907471710489
5,-1.243450e-13,-2.737810e-13,-8.739676e-13,0.884551836136517,0.884551836200175,2.25566427696241,2.25566427671358,1.33907471710489,1.33907471806113
6,-1.776357e-15,6.661338e-16,-7.105427e-15,0.884551836200175,0.884551836200170,2.25566427671358,2.25566427671360,1.33907471806113,1.33907471806111
7,1.776357e-15,1.110223e-15,7.105427e-15,0.884551836200170,0.884551836200170,2.25566427671360,2.25566427671360,1.33907471806111,1.33907471806111
8,8.881784e-16,8.881784e-16,0.000000e+00,0.884551836200170,0.884551836200170,2.25566427671360,2.25566427671360,1.33907471806111,1.33907471806111
9,8.881784e-16,8.881784e-16,0.000000e+00,0.884551836200170,0.884551836200170,2.25566427671360,2.25566427671360,1.33907471806111,1.33907471806111
